In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from sklearn.metrics import mean_squared_error
from scipy.ndimage.interpolation import shift
from math import sin, cos, sqrt, atan2, radians 
import dateutil
from pathlib import Path


Bad key "text.kerning_factor" on line 4 in
/opt/anaconda3/envs/bigdatalab_cpu_202101/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.2/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
status_path = Path('../filtered_status.csv')
stations_path = Path('../station.csv')

In [ ]:
status_df = pd.read_csv(status_path, parse_dates=['time'])
stations_df = pd.read_csv(stations_path)

In [4]:
status_df

,station_id,bikes_available,docks_available,time
0,2,2,25,2013-08-29 12:06:01
1,2,2,25,2013-08-29 12:07:01
2,2,2,25,2013-08-29 12:08:01
3,2,2,25,2013-08-29 12:09:01
4,2,2,25,2013-08-29 12:10:01
...,...,...,...,...
71977905,84,8,7,2015-08-31 23:55:02
71977906,84,8,7,2015-08-31 23:56:01
71977907,84,8,7,2015-08-31 23:57:02
71977908,84,8,7,2015-08-31 23:58:02


In [5]:
stations_df

,id,name,lat,long,dock_count,city,installation_date
0,2,San Jose Diridon Caltrain Station,37.329732,-121.901782,27,San Jose,8/6/2013
1,3,San Jose Civic Center,37.330698,-121.888979,15,San Jose,8/5/2013
2,4,Santa Clara at Almaden,37.333988,-121.894902,11,San Jose,8/6/2013
3,5,Adobe on Almaden,37.331415,-121.893200,19,San Jose,8/5/2013
4,6,San Pedro Square,37.336721,-121.894074,15,San Jose,8/7/2013
...,...,...,...,...,...,...,...
65,77,Market at Sansome,37.789625,-122.400811,27,San Francisco,8/25/2013
66,80,Santa Clara County Civic Center,37.352601,-121.905733,15,San Jose,12/31/2013
67,82,Broadway St at Battery St,37.798541,-122.400862,15,San Francisco,1/22/2014
68,83,Mezes Park,37.491269,-122.236234,15,Redwood City,2/20/2014


## Ottengo gli id delle stazioni per ogni città diversa

In [ ]:
cities = stations_df['city'].unique()
cities

In [ ]:
city_stations = {}

for city in cities:
    city_stations[city] = stations_df[stations_df['city'] == city]['id'].unique()

city_stations

## Controllo il primo e ultimo timestamp per le città in modo tale da controllare se ci sono dei periodi sfalsati

In [ ]:
stations_id = city_stations['San Jose']
for id_station in stations_id:
    min_time = status_df[status_df['station_id']==id_station]['time'].min()
    max_time = status_df[status_df['station_id']==id_station]['time'].max()
    print(f"STATION {id_station}: min {min_time}, max {max_time}\n")


Già da questa città possiamo vedere che ci sono stazioni con dei periodi sfalsati. Bisognerà quindi effettuare un merge sul time.

## Ottengo per ogni stazione una tabella che contiene per riga un timestamp e per colonne il numero di bici disponibili al tempo corrente e ai 4 istanti di tempo precedenti. Dopo di che, effettuo il merge per tutte le tabelle di tutte le stazioni appartenenti ad una certa città.

In [ ]:
interval=60
window_width = 5

cities_status_dfs = {}
cities_correaltion_dfs = {}

for city in cities:
    merged_df = pd.DataFrame()
    stations_id = city_stations[city]
    for id_station in stations_id:
        new_df = pd.DataFrame()
        single_station_df = status_df[status_df['station_id']==id_station][['time', 'bikes_available']]
        windowing_df =  single_station_df.resample(f"{interval}T", on = 'time').mean()
        #resetto l'indice
        windowing_df  = windowing_df.reset_index()
        new_df['time'] = windowing_df['time']
        for i in reversed(range(window_width)):
            new_df[f'{id_station}_T{i}'] =  windowing_df['bikes_available'].shift(periods=i)
        if len(merged_df) == 0:
            merged_df = new_df.copy(deep=True)
            continue
        merged_df = pd.merge(merged_df, new_df, how='outer', on='time')
    
    city_str= city.replace(" ", "")
    cities_status_dfs[city]= merged_df
    #salvo il dataframe ottenuto contentente le informazioni sullo stato delle stazioni
    merged_df.to_csv(f"./Analysis_results/{interval}_{window_width}/{city_str}_status.csv", index=False)

    city_correaltion = merged_df.corr()
    cities_correaltion_dfs[city]= city_correaltion
    # salvo il dataframe rappresentante la matrice di correlazione
    city_correaltion.to_csv(f"./Analysis_results/{interval}_{window_width}/Correlation_{city_str}_status.csv")
    
    f = plt.figure(figsize=(19, 15))
    plt.matshow(merged_df.corr(), fignum=f.number)
    plt.xticks(range(merged_df.select_dtypes(['number']).shape[1]), merged_df.select_dtypes(['number']).columns, fontsize=14, rotation=90)
    plt.yticks(range(merged_df.select_dtypes(['number']).shape[1]), merged_df.select_dtypes(['number']).columns, fontsize=14)
    cb = plt.colorbar()
    cb.ax.tick_params(labelsize=14)
    plt.title(f'Correlation Matrix {city_str} {interval} min, width {window_width}', fontsize=16);
    # salvo l'immagine conenente la matrice di correlazione
    plt.savefig(f'./Analysis_results/{interval}_{window_width}/CorrelationMatrix_{city_str}_{interval}_{window_width}')

# single_station_df

In [ ]:
# windowing_df

In [ ]:
# merged_df.head()

In [ ]:
#PLOTTING

for city in cities:
    current_df = cities_status_dfs[city]
    f = plt.figure(figsize=(19, 15))
    plt.matshow(current_df.corr(), fignum=f.number)
    plt.xticks(range(current_df.select_dtypes(['number']).shape[1]), current_df.select_dtypes(['number']).columns, fontsize=14, rotation=90)
    plt.yticks(range(current_df.select_dtypes(['number']).shape[1]), current_df.select_dtypes(['number']).columns, fontsize=14)
    cb = plt.colorbar()
    cb.ax.tick_params(labelsize=14)
    plt.title(f'Correlation Matrix {city} {interval} min, width {window_width}', fontsize=16);

In [ ]:

cities_status_dfs["Palo Alto"].corr().style.background_gradient(cmap='copper_r')


## Calcolo la distanza tra le diverse stazioni

In [6]:
#function to retrieve distance between 2 stations
def getDistance(lat_a, long_a, lat_b, long_b):
    # approximate radius of earth in km
    R = 6373.0    
    lat1=radians(lat_a)
    lat2=radians(lat_b)
    lon1=radians(long_a)
    lon2=radians(long_b)
    
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [7]:
coordinates_stations_df = stations_df[["id", "lat", "long"]]
records = coordinates_stations_df.to_records(index=False)
list_coo = list(records)
# print(list_coo)

voc_distances={}
for i in range(len(list_coo)):
    for j in range(i+1,len(list_coo)):
        station1=list_coo[i][0]
        station2=list_coo[j][0]
        lat_i=float(list_coo[i][1])
        long_i=float(list_coo[i][2])
        lat_j=float(list_coo[j][1])
        long_j=float(list_coo[j][2])
        distance=getDistance(lat_i, long_i, lat_j, long_j)
        id_stations=str(station1)+' '+str(station2)
        voc_distances[id_stations]=distance


In [9]:
voc_distances['27 28']

0.7325708444979485